In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import json

In [34]:
#page = requests.get("https://www.ikb.at/privat/baeder/aktuelle-auslastung/")
page = requests.get("http://sas.ikb.at/ks/baeder_auslastung.aspx?segment=privat")
soup = BeautifulSoup(page.content, 'lxml')

try:
    #Amras
    amras = soup.find(id="cphInhalt_pnlHBA_Bad").find(class_="info-block").span.span.text
except AttributeError:
    amras = 100
try:
    #O-Dorf
     o_dorf = soup.find(id="cphInhalt_pnlHBO_Bad").find(class_="info-block").span.span.text
except AttributeError:
    o_dorf = 100
try:
    #Hötting
    hoetting = soup.find(id="cphInhalt_pnlHBH_Bad").find(class_="info-block").span.span.text
except AttributeError:
    hoetting = 100
    

# Current weather
OWMID = os.environ['OWMID']
weather_page = requests.get('http://api.openweathermap.org/data/2.5/weather?q=Innsbruck,%20ATE&appid=' + OWMID)
weather = json.loads(weather_page.content)
temp = weather['main']['temp']
humidity = int(weather['main']['humidity'])/100
clouds = int(weather['clouds']['all'])/100

# Current time
timestamp = datetime.now()
year = timestamp.strftime('%Y')
month = timestamp.strftime('%m')
day = timestamp.strftime('%d')
hour = timestamp.strftime('%H')
minute = timestamp.strftime('%M')
day = timestamp.weekday()

row = [{'temp':temp, 'humidity':humidity,'year':year, 'clouds':clouds, 'timestamp':timestamp,'day':day, 'hour':hour, 'minute':minute, 'month':month, 'day':day,  'amras': int(amras)/100, 'o_dorf': int(o_dorf)/100, 'hoetting': int(hoetting)/100} ]
df = pd.DataFrame.from_dict(row)

# Write to CSV
if not os.path.isfile('ikb.csv'):
    df.to_csv('ikb.csv', header ='column_names', index=False)
else:
    df.to_csv('ikb.csv', mode = 'a', header=False, index=False)

In [35]:
new = pd.read_csv('ikb.csv')
new.head(10)

,amras,clouds,day,hoetting,hour,humidity,minute,month,o_dorf,temp,timestamp,year
0,0.02,0.75,4,0.45,19,0.93,45,4,0.44,275.33,2017-04-28 19:45:48.338225,2017
1,0.02,0.75,4,0.45,20,0.86,1,4,0.41,275.38,2017-04-28 20:01:17.441462,2017
2,0.02,0.75,4,0.45,20,0.86,4,4,0.41,275.38,2017-04-28 20:04:18.648576,2017
3,0.01,0.75,5,0.01,1,0.93,6,4,0.01,274.97,2017-04-29 01:06:40.938171,2017
4,1.00,0.75,5,1.00,1,0.93,7,4,1.00,274.97,2017-04-29 01:07:13.171570,2017
5,1.00,0.75,5,1.00,1,0.93,12,4,1.00,274.97,2017-04-29 01:12:09.659723,2017
6,1.00,0.75,5,1.00,1,0.93,12,4,1.00,274.97,2017-04-29 01:12:15.981754,2017
7,1.00,0.75,5,1.00,1,0.93,18,4,1.00,274.94,2017-04-29 01:18:49.082905,2017
8,0.17,0.40,5,0.29,9,0.80,58,4,1.00,276.40,2017-04-29 09:58:46.393935,2017
